In [2]:
import sys 
sys.path.append('..')
from utils.dataset import load_harmful_ins_dataset
from extract_prompt import full_policy_prompt_order

import os 
import json 
import re
import datetime
from openai import OpenAI
api_key = ''


In [ ]:
def parse_resp(response):
    if "#Theanswer:" in response :
        response = response.split("#Theanswer:")[1].strip()
        response = response.replace('"','').replace("'","")
        response = response[response.find('[')+1:response.find(']')]
        word_list = re.split('\s|,[\s]*',response)
        return word_list
    else :
        return []

In [ ]:
response_data = {}

client = OpenAI(api_key=api_key)

file_name  = "gpt4_gpt_policy"
store_dir = './batch'
map_dir = './map'
error_dir = './error'
result_dir = './result'

model = "gpt-4o-mini-2024-07-18"
#model = "gpt-4o-2024-08-06"
os.makedirs(store_dir,exist_ok=True)
os.makedirs(map_dir,exist_ok=True)
os.makedirs(error_dir,exist_ok=True)
os.makedirs(result_dir,exist_ok=True)


dataset_list = ['custom','hexphi']
ins_list = [] 
for dataset in dataset_list:
    ins_list += load_harmful_ins_dataset(dataset)

#ins_list = ins_list[:20]

batch_file = f'{store_dir}/{file_name}-{model}-{datetime.datetime.now()}.jsonl'
if os.path.isfile(batch_file):
    os.remove(batch_file)


map_file = f'{map_dir}/ins_map.json'


In [ ]:
request = {
  "custom_id": "request-1",
  "method": "POST",
  "url": "/v1/chat/completions",
  "body": {
    "model" : f"{model}",
    "temperature" : 0.0,
    "messages": [
      {
        "role": "system",
        "content": "You are a helpful assistant."
      },
      {
        "role": "user",
        "content": "Hello world!"
      }
    ],
    "max_tokens": 4096,
    
  }
}

In [ ]:
def get_message(prompt):
    return [
      {
        "role": "system",
        "content": "You are a helpful assistant."
      },
      {
        "role": "user",
        "content": prompt
      }
    ]

In [ ]:
def retrive_error_batch(file_name, batch_id):
    error_custom_id_list = []
    f = open(file_name,'r')

    while True: 
        data = f.readline()
        if not data:
            break
        resp = json.loads(data)    
        error_custom_id_list.append(resp['custom_id'])

    f.close()

    f_batch = open(batch_file,'r')
    error_req_list = []
    while True: 
        data = f_batch.readline()
        if not data:
            break
        resp = json.loads(data)    
        if resp['custom_id'] in error_custom_id_list:
            error_req_list.append(data)
    
    f_batch.close()

    ### write error batch to a file
    error_file_name = f'{error_dir}/{batch_id}_error.jsonl'

    if os.path.isfile(error_file_name):
        os.remove(error_file_name)

    for req in error_req_list :
        f_error = open(error_file_name,'a+')
        f_error.write(json.dumps(req))
        f_error.write('\n')
        f_error.close()
            
    return error_file_name
    

In [ ]:

def update(output_file):
    global response_data
    output = open(output_file,'r')
    while True: 
        data = output.readline()
        if not data:
            break
        resp = json.loads(data)

        custom_id = resp['custom_id']
        content = resp['response']['body']['choices'][0]['message']['content']
        print(content)
        word_list = parse_resp(content)
        response_data[custom_id] = word_list

    output.close()

In [ ]:
map_dict = []

for i, instruction in enumerate(ins_list):
    map_data = {}
    map_data['instruction'] = instruction
    custom_id = f'ins_{i}'
    map_data['custom_id'] = custom_id
    
    prompt = full_policy_prompt_order(instruction)
    print(prompt)
    message = get_message(prompt)

    map_data['prompt'] = prompt

    request['body']['messages'] = message
    request['custom_id'] = custom_id
    
    batch_f = open(batch_file,'a+')
    batch_f.write(json.dumps(request))
    batch_f.write('\n')
    batch_f.close()

    map_dict.append(map_data)
    map_f = open(map_file,'w')
    map_f.write(json.dumps(map_dict,indent=4))
    map_f.close()

    

In [ ]:

batch_input_file = client.files.create(
  file=open(batch_file, "rb"),
  purpose="batch"
)


In [ ]:
batch_input_file_id = batch_input_file.id

batch_response = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": f"extractor task for harmful instruction"
    }
)

In [ ]:
batch_id = batch_response.id
status = client.batches.retrieve(batch_id).status
assert status == 'completed'

output_file_name = f'{store_dir}/extract_result_{datetime.datetime.now()}_output.jsonl'
error_file_name = f'{store_dir}/extract_result_{datetime.datetime.now()}_error.jsonl'

success_file_id = client.batches.retrieve(batch_id).output_file_id
client.files.content(success_file_id).write_to_file(output_file_name)

update(output_file_name)

retrive = False 

if client.batches.retrieve(batch_id).error_file_id != None:
    print("Need to do one more error handle")
    error_file_id = client.batches.retrieve(batch_id).error_file_id
    client.files.content(error_file_id).write_to_file(error_file_name)
    error_batch_file = retrive_error_batch(error_file_name, batch_id)
    print(error_batch_file)

    if retrive :
        batch_input_file = client.files.create(
            file=open(error_batch_file, "rb"),
            purpose="batch"
        )     
        batch_input_file_id = batch_input_file.id
        batch_response = client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={
            "description": f"extractor task for harmful instruction"
            }
        )

    



In [ ]:
print(len(response_data))
for key in response_data:
    print(response_data[key])
    if response_data[key] == [] :
        pass
        #print(key)

In [ ]:

f_map = open(map_file,'r')
map_data = json.loads(f_map.read())
f_map.close()
file_store = f'{result_dir}/harmful_word_{model}_{datetime.datetime.now()}.json'


for i in range(len(map_data)):

    custom_id = map_data[i]['custom_id']
    map_data[i]['word_list'] = response_data[custom_id]  
    with open(file_store,'w') as f2:
        json.dump(map_data,f2,indent=4)
